# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments  \
0  Dec 20 2022  7:44AM  253451        20   8925639   
1  Dec 20 2022  7:44AM  253451        20   8925640   
2  Dec 20 2022  7:44AM  253451        20   8925641   
3  Dec 20 2022  7:44AM  253451        20   8925642   
4  Dec 20 2022  7:44AM  253451        20   8925643   
5  Dec 20 2022  7:44AM  253451        20   8925644   
6  Dec 20 2022  7:44AM  253451        20   8925645   
7  Dec 20 2022  7:44AM  253451        20   8925646   
8  Dec 20 2022  7:44AM  253451        20   8925647   
9  Dec 19 2022  4:58PM  253449        15   8922341   

                          Customer ShipmentStatus  
0                   Exact-Rx, Inc.       Released  
1                   Exact-Rx, Inc.       Released  
2                   Exact-Rx, Inc.       Released  
3                   Exact-Rx, Inc.       Released  
4                   Exact-Rx, Inc.       Released  
5                   Exact-Rx, Inc.       Released  
6                   Exact-Rx, Inc.       Released  
7                   Exact-Rx, Inc.       Released  
8                   Exact-Rx, Inc.       Released  
9  Brookfield Pharmaceuticals, LLC       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
23  253445       Released          2
24  253446       Released         15
25  253448       Released          1
26  253449       Released         30
27  253451       Released          9

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Released
id                      
253445                 2
253446                15
253448                 1
253449                30
253451                 9

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Released
id                      
253377                 1
253382                 1
253383                 1
253384                 3
253387                 3

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus  Released
id                      
253377                 1
253382                 1
253383                 1
253384                 3
253387                 3

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Released
0               253377         1
1               253382         1
2               253383         1
3               253384         3
4               253387         3

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus      id  Released
0               253377         1
1               253382         1
2               253383         1
3               253384         3
4               253387         3

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id          int64
Released    int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Dec 20 2022  7:44AM  253451        20                   Exact-Rx, Inc.
9   Dec 19 2022  4:58PM  253449        15  Brookfield Pharmaceuticals, LLC
39  Dec 19 2022  4:56PM  253448        15      Virtus Pharmaceuticals, LLC
40  Dec 19 2022  4:55PM  253446        15            Alliance Pharma, Inc.
55  Dec 19 2022  4:50PM  253445        10     Snap Medical Industries, LLC
57  Dec 19 2022  4:48PM  253444        22                NBTY Global, Inc.
58  Dec 19 2022  4:47PM  253442        10                ISDIN Corporation
71  Dec 19 2022  4:47PM  253441        10                ISDIN Corporation
81  Dec 19 2022  4:39PM  253440        10                ISDIN Corporation
85  Dec 19 2022  4:37PM  253437        10                ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Dec 20 2022  7:44AM  253451        20                   Exact-Rx, Inc.   
1  Dec 19 2022  4:58PM  253449        15  Brookfield Pharmaceuticals, LLC   
2  Dec 19 2022  4:56PM  253448        15      Virtus Pharmaceuticals, LLC   
3  Dec 19 2022  4:55PM  253446        15            Alliance Pharma, Inc.   
4  Dec 19 2022  4:50PM  253445        10     Snap Medical Industries, LLC   
5  Dec 19 2022  4:48PM  253444        22                NBTY Global, Inc.   
6  Dec 19 2022  4:47PM  253442        10                ISDIN Corporation   
7  Dec 19 2022  4:47PM  253441        10                ISDIN Corporation   
8  Dec 19 2022  4:39PM  253440        10                ISDIN Corporation   
9  Dec 19 2022  4:37PM  253437        10                ISDIN Corporation   

   Released  
0         9  
1        30  
2         1  
3        15  
4         2  
5         1  
6        13  
7        10  
8         4  
9         3

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Dec 20 2022  7:44AM  253451        20                   Exact-Rx, Inc.   
1  Dec 19 2022  4:58PM  253449        15  Brookfield Pharmaceuticals, LLC   
2  Dec 19 2022  4:56PM  253448        15      Virtus Pharmaceuticals, LLC   
3  Dec 19 2022  4:55PM  253446        15            Alliance Pharma, Inc.   
4  Dec 19 2022  4:50PM  253445        10     Snap Medical Industries, LLC   
5  Dec 19 2022  4:48PM  253444        22                NBTY Global, Inc.   
6  Dec 19 2022  4:47PM  253442        10                ISDIN Corporation   
7  Dec 19 2022  4:47PM  253441        10                ISDIN Corporation   
8  Dec 19 2022  4:39PM  253440        10                ISDIN Corporation   
9  Dec 19 2022  4:37PM  253437        10                ISDIN Corporation   

   Released  
0         9  
1        30  
2         1  
3        15  
4         2  
5         1  
6        13  
7        10  
8         4  
9         3

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Dec 20 2022  7:44AM  253451        20                   Exact-Rx, Inc.   
1  Dec 19 2022  4:58PM  253449        15  Brookfield Pharmaceuticals, LLC   
2  Dec 19 2022  4:56PM  253448        15      Virtus Pharmaceuticals, LLC   
3  Dec 19 2022  4:55PM  253446        15            Alliance Pharma, Inc.   
4  Dec 19 2022  4:50PM  253445        10     Snap Medical Industries, LLC   

   Released  
0         9  
1        30  
2         1  
3        15  
4         2

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet
Column Completed does not exist yet


Date      id Warehouse                         Customer  \
0  Dec 20 2022  7:44AM  253451        20                   Exact-Rx, Inc.   
1  Dec 19 2022  4:58PM  253449        15  Brookfield Pharmaceuticals, LLC   
2  Dec 19 2022  4:56PM  253448        15      Virtus Pharmaceuticals, LLC   
3  Dec 19 2022  4:55PM  253446        15            Alliance Pharma, Inc.   
4  Dec 19 2022  4:50PM  253445        10     Snap Medical Industries, LLC   

   Released  
0         9  
1        30  
2         1  
3        15  
4         2

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Dec 20 2022  7:44AM  253451        20                   Exact-Rx, Inc.   
1  Dec 19 2022  4:58PM  253449        15  Brookfield Pharmaceuticals, LLC   
2  Dec 19 2022  4:56PM  253448        15      Virtus Pharmaceuticals, LLC   
3  Dec 19 2022  4:55PM  253446        15            Alliance Pharma, Inc.   
4  Dec 19 2022  4:50PM  253445        10     Snap Medical Industries, LLC   

   Released  
0         9  
1        30  
2         1  
3        15  
4         2

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released
Warehouse                   
10         3041014        55
15         1774010        56
19          760228         7
20          253451         9
21         1013590         4
22          253444         1

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released
0        10  3041014        55
1        15  1774010        56
2        19   760228         7
3        20   253451         9
4        21  1013590         4
5        22   253444         1

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released
0        10        55
1        15        56
2        19         7
3        20         9
4        21         4
5        22         1

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released     55
1        15  Released     56
2        19  Released      7
3        20  Released      9
4        21  Released      4

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse  10  15  19  20  21  22
Status                           
Released   55  56   7   9   4   1

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse    Status  10  15  19  20  21  22
0          Released  55  56   7   9   4   1

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status  10  15  19  20  21  22
0  Released  55  56   7   9   4   1

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()